In [12]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function orig_is_subset(X,Y)
    for i = 1:size(X)[1]
        if !( issubset(X[i],Y[i]) )
            return false
        end
    end
    return true
end

function orig_intersect(X,Y)
    L = []
    for i = 1:size(X)[1]
        push!(L, intersect(X[i],Y[i]))
        # typeof(intersect(X[i],Y[i])) => Interval{Float64}
    end
    return L
end

function orig_is_empty(X,Y)
    for i = 1:size(X)[1]
        if ( isempty(intersect(X[i],Y[i])) )
            return true
        end
    end
    return false
end

function orig_devide(X)
    #最も長い辺を二等分する
    max_diam=(0..0)
    max_index=0
    X1,X2=[],[]
    for (i,x) in enumerate(X)
        if diam(x)>max_diam
            max_diam=diam(x)
            max_index=i
        end
    end
    
    if diam(X[max_index])<10^(-8)
        return false
    end
    
    X1=X*1 #値をコピーする
    X2=X*1 #値をコピーする
    X1[max_index]= intersect(X[max_index],X[max_index]-radius(X[max_index]))
    X2[max_index]= intersect(X[max_index],X[max_index]+(diam(X[max_index])-radius(X[max_index])))
    return X1,X2
end


function allsol(F,X)
    cnt=0
    L = [X] #調査する区間のリスト
    S = [] #見つかった解のリスト
    
    while (size(L)[1] != 0)
        
        X = popfirst!(L) #Lから区間を一つ取り出す
        
        if !( orig_is_subset([(0. ..0.),(0. ..0.)], F(X)) ) 
            print(X)
            println(":非存在条件を満たす")
            continue
        end
        
        c = mid.(X)
        j = ForwardDiff.jacobian(F,X)
        
        if !( orig_is_subset([(0. ..0.),(0. ..0.)], F(c)+j*(X-c)) )
            print(X)
            println(":非存在条件を満たす")
            continue
        end
    
        tmp = ForwardDiff.jacobian(F,c)
        
        R = tmp
        try
            R = inv(tmp)
        catch e
            try
                X1,X2 = orig_devide(X)
                push!(L, X1)
                push!(L, X2)
            catch e
                println(e)
            end
            print(X)
            println(":逆行列の演算に失敗")
            continue
        end
        
        M = Matrix{Float64}(I,size(R))-R*j
        K = c -R*F(c)+M*(X-c)
        @show K
        @show X
        
        if (orig_is_empty(K,X))
            print(X)
            println(":解なし")
            continue #解なし
        end
        
        if (orig_is_subset(K,X))　#解あり
            push!(S, K) #解のリストに追加
            print(X)
            println(":解あり")
            continue
        else
            print(K)
            println(X)
        end
        
        try
            print(X)
            println(":未知")
            X1,X2 = orig_devide(orig_intersect(K,X))　#K and X を2分割
            push!(L, X1)
            push!(L, X2)
        catch e
            println(e)
        end
    end
    @show L
    @show S
    return S
end

X = [(0.1 ..1.9),(-2. .. 2.)]

f(x, y) = x1^2 + y^2 - 1
g(x, y) = x1^2 - y^4

#f((x1,x2)) = x1 + 2*x2 +1
#g((x1,x2)) = x1^2 + 2*(x2^2) -1

# f((x1,x2)) = x1 + 2*x2 +1
# g((x1,x2)) = x1^2 + 2*(x2^2) -1

F((x,y))=[f((x,y));g((x,y))]

li=allsol(F,X)

for i in li
    @show i
end

# 境界に解がのっているとき、区間拡張？
# 重解　＝＞区間幅に制限をつける

Interval{Float64}[[0.0999999, 1.90001], [-2, 2]]:逆行列の演算に失敗
Interval{Float64}[[0.0999999, 1.90001], [-2, 0]]:逆行列の演算に失敗
Interval{Float64}[[0.0999999, 1.90001], [0, 2]]:逆行列の演算に失敗
Interval{Float64}[[0.0999999, 1.90001], [-2, -1]]:非存在条件を満たす
Interval{Float64}[[0.0999999, 1.90001], [-1, 0]]:逆行列の演算に失敗
Interval{Float64}[[0.0999999, 1.90001], [0, 1]]:逆行列の演算に失敗
Interval{Float64}[[0.0999999, 1.90001], [1, 2]]:非存在条件を満たす
K = Interval{Float64}[[0.324999, 1.22501], [-0.450001, 1.35001]]
X = Interval{Float64}[[0.0999999, 1], [-1, 0]]
Interval{Float64}[[0.324999, 1.22501], [-0.450001, 1.35001]]Interval{Float64}[[0.0999999, 1], [-1, 0]]
Interval{Float64}[[0.0999999, 1], [-1, 0]]:未知
K = Interval{Float64}[[0.774999, 1.67501], [-1.35001, 0.450001]]
X = Interval{Float64}[[1, 1.90001], [-1, 0]]
Interval{Float64}[[0.774999, 1.67501], [-1.35001, 0.450001]]Interval{Float64}[[1, 1.90001], [-1, 0]]
Interval{Float64}[[1, 1.90001], [-1, 0]]:未知
K = Interval{Float64}[[0.324999, 1.22501], [-0.450001, 1.35001]]
X = Inte

Interval{Float64}[[1, 1.0534], [-0.189844, -0.0949218]]:解なし
K = Interval{Float64}[[1.03114, 1.04895], [-0.0978882, -0.0622924]]
X = Interval{Float64}[[1.05339, 1.10679], [-0.189844, -0.0949218]]
Interval{Float64}[[1.05339, 1.10679], [-0.189844, -0.0949218]]:解なし
K = Interval{Float64}[[0.986651, 1.04005], [-0.0800904, 0.0266968]]
X = Interval{Float64}[[1, 1.0534], [-0.0949219, 0]]
Interval{Float64}[[0.986651, 1.04005], [-0.0800904, 0.0266968]]Interval{Float64}[[1, 1.0534], [-0.0949219, 0]]
Interval{Float64}[[1, 1.0534], [-0.0949219, 0]]:未知
Interval{Float64}[[1.05339, 1.10679], [-0.0949219, 0]]:非存在条件を満たす
Interval{Float64}[[0.893212, 0.946607], [0, 0.0949219]]:非存在条件を満たす
K = Interval{Float64}[[0.959954, 1.01335], [-0.0266968, 0.0800904]]
X = Interval{Float64}[[0.946606, 1], [0, 0.0949219]]
Interval{Float64}[[0.959954, 1.01335], [-0.0266968, 0.0800904]]Interval{Float64}[[0.946606, 1], [0, 0.0949219]]
Interval{Float64}[[0.946606, 1], [0, 0.0949219]]:未知
K = Interval{Float64}[[0.951055, 0.96885

K = Interval{Float64}[[0.999967, 1.0001], [-0.000197978, 6.59924e-05]]
X = Interval{Float64}[[1, 1.00014], [0, 0.00017598]]
Interval{Float64}[[0.999967, 1.0001], [-0.000197978, 6.59924e-05]]Interval{Float64}[[1, 1.00014], [0, 0.00017598]]
Interval{Float64}[[1, 1.00014], [0, 0.00017598]]:未知
Interval{Float64}[[1.00013, 1.00027], [0, 0.00017598]]:非存在条件を満たす
Interval{Float64}[[0.999901, 0.999951], [-6.59925e-05, 0]]:非存在条件を満たす
K = Interval{Float64}[[0.999962, 1.00002], [-2.47472e-05, 7.42415e-05]]
X = Interval{Float64}[[0.99995, 1], [-6.59925e-05, 0]]
Interval{Float64}[[0.999962, 1.00002], [-2.47472e-05, 7.42415e-05]]Interval{Float64}[[0.99995, 1], [-6.59925e-05, 0]]
Interval{Float64}[[0.99995, 1], [-6.59925e-05, 0]]:未知
Interval{Float64}[[1, 1.00317], [-0.0126706, -0.00844702]]:非存在条件を満たす
K = Interval{Float64}[[1.00184, 1.00291], [-0.00580734, -0.00369557]]
X = Interval{Float64}[[1.00316, 1.00634], [-0.0126706, -0.00844702]]
Interval{Float64}[[1.00316, 1.00634], [-0.0126706, -0.00844702]]:解なし

X = Interval{Float64}[[0.999498, 1], [0.000890897, 0.0017818]]
Interval{Float64}[[0.999498, 1], [0.000890897, 0.0017818]]:解なし
K = Interval{Float64}[[0.999999, 1.00001], [-1.46821e-06, 4.89339e-07]]
X = Interval{Float64}[[1, 1.00001], [0, 1.30502e-06]]
Interval{Float64}[[0.999999, 1.00001], [-1.46821e-06, 4.89339e-07]]Interval{Float64}[[1, 1.00001], [0, 1.30502e-06]]
Interval{Float64}[[1, 1.00001], [0, 1.30502e-06]]:未知
Interval{Float64}[[1, 1.00001], [0, 1.30502e-06]]:非存在条件を満たす
Interval{Float64}[[0.999999, 1], [-4.89344e-07, 0]]:非存在条件を満たす
K = Interval{Float64}[[0.999999, 1.00001], [-1.83604e-07, 5.50486e-07]]
X = Interval{Float64}[[0.999999, 1], [-4.89344e-07, 0]]
Interval{Float64}[[0.999999, 1.00001], [-1.83604e-07, 5.50486e-07]]Interval{Float64}[[0.999999, 1], [-4.89344e-07, 0]]
Interval{Float64}[[0.999999, 1], [-4.89344e-07, 0]]:未知
K = Interval{Float64}[[0.999906, 1.00029], [-0.000563772, 0.000187924]]
X = Interval{Float64}[[1, 1.00038], [-0.000751695, -0.000375847]]
Interval{Float64

Interval{Float64}[[0.999985, 1.00005], [-8.91904e-05, 2.97302e-05]]Interval{Float64}[[1, 1.00006], [-7.92803e-05, -3.96401e-05]]
Interval{Float64}[[1, 1.00006], [-7.92803e-05, -3.96401e-05]]:未知
K = Interval{Float64}[[0.999985, 1.00005], [-8.91904e-05, 2.97302e-05]]
X = Interval{Float64}[[1, 1.00006], [-3.96402e-05, 0]]
Interval{Float64}[[0.999985, 1.00005], [-8.91904e-05, 2.97302e-05]]Interval{Float64}[[1, 1.00006], [-3.96402e-05, 0]]
Interval{Float64}[[1, 1.00006], [-3.96402e-05, 0]]:未知
K = Interval{Float64}[[0.999955, 1.00002], [-2.97302e-05, 8.91904e-05]]
X = Interval{Float64}[[0.99994, 1], [0, 3.96402e-05]]
Interval{Float64}[[0.999955, 1.00002], [-2.97302e-05, 8.91904e-05]]Interval{Float64}[[0.99994, 1], [0, 3.96402e-05]]
Interval{Float64}[[0.99994, 1], [0, 3.96402e-05]]:未知
K = Interval{Float64}[[0.999955, 1.00002], [-2.97302e-05, 8.91904e-05]]
X = Interval{Float64}[[0.99994, 1], [3.96401e-05, 7.92803e-05]]
Interval{Float64}[[0.999955, 1.00002], [-2.97302e-05, 8.91904e-05]]Interval

Interval{Float64}[[0.999998, 1.00001], [-1.32285e-06, 3.96859e-06]]Interval{Float64}[[0.999997, 1], [0, 1.76379e-06]]
Interval{Float64}[[0.999997, 1], [0, 1.76379e-06]]:未知
K = Interval{Float64}[[0.999998, 1.00001], [-1.32289e-06, 3.96855e-06]]
X = Interval{Float64}[[0.999997, 1], [1.76378e-06, 3.52758e-06]]
Interval{Float64}[[0.999998, 1.00001], [-1.32289e-06, 3.96855e-06]]Interval{Float64}[[0.999997, 1], [1.76378e-06, 3.52758e-06]]
Interval{Float64}[[0.999997, 1], [1.76378e-06, 3.52758e-06]]:未知
K = Interval{Float64}[[0.999998, 1], [1.54335e-06, 2.42526e-06]]
X = Interval{Float64}[[0.999997, 0.999999], [3.52757e-06, 5.29143e-06]]
Interval{Float64}[[0.999997, 0.999999], [3.52757e-06, 5.29143e-06]]:解なし
Interval{Float64}[[0.999998, 1], [3.52757e-06, 5.29143e-06]]:非存在条件を満たす
K = Interval{Float64}[[0.999999, 1.00001], [-1.48828e-06, 4.9596e-07]]
X = Interval{Float64}[[1, 1.00001], [-3.52756e-06, -1.76377e-06]]
Interval{Float64}[[1, 1.00001], [-3.52756e-06, -1.76377e-06]]:解なし
K = Interval{Flo

Interval{Float64}[[0.999999, 1], [-2.58579e-16, 2.33028e-08]]Interval{Float64}[[0.999999, 1], [1.68569e-08, 3.3714e-08]]
Interval{Float64}[[0.999999, 1], [1.68569e-08, 3.3714e-08]]:未知
K = Interval{Float64}[[0.999999, 1.00001], [-5.17798e-09, 1.25355e-08]]
X = Interval{Float64}[[1, 1.00001], [-1.77677e-08, -8.88381e-09]]
Interval{Float64}[[1, 1.00001], [-1.77677e-08, -8.88381e-09]]:解なし
K = Interval{Float64}[[1, 1.00001], [-1.6987e-08, -1.10824e-08]]
X = Interval{Float64}[[1, 1.00001], [-1.77677e-08, -8.88381e-09]]
Interval{Float64}[[1, 1.00001], [-1.77677e-08, -8.88381e-09]]:解なし
K = Interval{Float64}[[1, 1.00001], [-1.45796e-08, -4.96963e-17]]
X = Interval{Float64}[[1, 1.00001], [-8.88382e-09, 0]]
Interval{Float64}[[1, 1.00001], [-1.45796e-08, -4.96963e-17]]Interval{Float64}[[1, 1.00001], [-8.88382e-09, 0]]
Interval{Float64}[[1, 1.00001], [-8.88382e-09, 0]]:未知
BoundsError(false, 2)
Interval{Float64}[[1, 1.00001], [-8.88382e-09, 0]]:非存在条件を満たす
K = Interval{Float64}[[0.999999, 1], [-1.5687